In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=0.)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.38 sec - Loss 0.243613 - ACC 77.61% - ACC Mean 77.61% - AUC 85.66% - AUC Mean 85.66% - Deter 000
Ite 00050 - 0.82 sec - Loss 0.122702 - ACC 78.07% - ACC Mean 77.68% - AUC 84.32% - AUC Mean 84.01% - Deter 047
Ite 00053 - 0.96 sec - Loss 0.120403 - ACC 77.91% - ACC Mean 77.71% - AUC 84.30% - AUC Mean 83.87% - Deter 050
Early stop ite 53, rollback to correction of ite 3, whith acc of 77.07% and auc of 86.4%
OUR METHOD RUN 1/10 - acc: 77.07% +- 0.0% - auc: 86.4% +- 0.0%
Ite 00000 - 0.83 sec - Loss 0.247092 - ACC 78.02% - ACC Mean 78.02% - AUC 85.75% - AUC Mean 85.75% - Deter 000
Ite 00050 - 0.88 sec - Loss 0.130608 - ACC 78.19% - ACC Mean 78.16% - AUC 84.35% - AUC Mean 84.47% - Deter 050
Early stop ite 50, rollback to correction of ite 0, whith acc of 78.02% and auc of 85.75%
OUR METHOD RUN 2/10 - acc: 78.02% +- 0.0% - auc: 85.75% +- 0.0%
Ite 00000 - 0.85 sec - Loss 0.218782 - ACC 75.94% - ACC Mean 75.94% - AUC 85.19% - AUC Mean 85.19% - Deter 000
Ite 00050 - 0.82 sec - Loss 

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 76.55% +- 0.0% - auc: 81.39% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 57.91% +- 3.82% - auc: 64.49% +- 4.51%
SFIL RUN 2/10 - acc: 59.62% +- 4.91% - auc: 64.35% +- 2.34%
SFIL RUN 3/10 - acc: 57.07% +- 2.89% - auc: 64.07% +- 5.69%
SFIL RUN 4/10 - acc: 56.76% +- 1.91% - auc: 66.63% +- 4.01%
SFIL RUN 5/10 - acc: 57.83% +- 0.83% - auc: 67.43% +- 4.78%
SFIL RUN 6/10 - acc: 58.31% +- 1.14% - auc: 63.88% +- 1.85%
SFIL RUN 7/10 - acc: 57.84% +- 2.43% - auc: 66.31% +- 2.9%
SFIL RUN 8/10 - acc: 57.82% +- 1.16% - auc: 64.8% +- 1.49%
SFIL RUN 9/10 - acc: 59.79% +- 6.03% - auc: 64.84% +- 4.93%
SFIL RUN 10/10 - acc: 57.82% +- 1.99% - auc: 65.1% +- 5.03%
SFIL GLOBAL - acc: 58.08% +- 0.92% - auc: 65.19% +- 1.13%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 74.55% +- 2.98% - auc: 83.83% +- 2.64%
PFIL RUN 2/10 - acc: 72.98% +- 3.11% - auc: 82.08% +- 2.63%
PFIL RUN 3/10 - acc: 75.37% +- 3.4% - auc: 83.54% +- 2.77%
PFIL RUN 4/10 - acc: 73.26% +- 1.59% - auc: 83.68% +- 2.09%
PFIL RUN 5/10 - acc: 73.98% +- 3.88% - auc: 84.42% +- 3.12%
PFIL RUN 6/10 - acc: 74.79% +- 3.91% - auc: 82.97% +- 2.87%
PFIL RUN 7/10 - acc: 74.31% +- 3.3% - auc: 84.02% +- 3.41%
PFIL RUN 8/10 - acc: 73.34% +- 3.61% - auc: 84.5% +- 4.52%
PFIL RUN 9/10 - acc: 74.17% +- 3.27% - auc: 81.03% +- 1.22%
PFIL RUN 10/10 - acc: 74.48% +- 2.55% - auc: 82.8% +- 3.3%
PFIL GLOBAL - acc: 74.12% +- 0.71% - auc: 83.28% +- 1.03%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 63.79% +- 5.47% - auc: 69.28% +- 6.42%
SPOL RUN 2/10 - acc: 61.39% +- 2.91% - auc: 66.62% +- 3.18%
SPOL RUN 3/10 - acc: 63.85% +- 6.5% - auc: 68.7% +- 4.22%
SPOL RUN 4/10 - acc: 65.08% +- 7.5% - auc: 69.4% +- 7.15%
SPOL RUN 5/10 - acc: 68.33% +- 5.26% - auc: 73.52% +- 5.09%
SPOL RUN 6/10 - acc: 64.64% +- 3.82% - auc: 68.93% +- 2.35%
SPOL RUN 7/10 - acc: 64.59% +- 6.78% - auc: 67.55% +- 7.58%
SPOL RUN 8/10 - acc: 69.37% +- 5.24% - auc: 74.12% +- 5.94%
SPOL RUN 9/10 - acc: 67.4% +- 6.76% - auc: 72.53% +- 3.51%
SPOL RUN 10/10 - acc: 66.77% +- 6.24% - auc: 70.27% +- 7.42%
SPOL GLOBAL - acc: 65.52% +- 2.29% - auc: 70.09% +- 2.39%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 73.57% +- 3.37% - auc: 83.3% +- 4.56%
PPOL RUN 2/10 - acc: 73.4% +- 3.66% - auc: 81.65% +- 4.64%
PPOL RUN 3/10 - acc: 77.09% +- 2.98% - auc: 83.88% +- 2.7%
PPOL RUN 4/10 - acc: 74.61% +- 3.81% - auc: 84.28% +- 2.32%
PPOL RUN 5/10 - acc: 74.65% +- 2.6% - auc: 83.87% +- 1.68%
PPOL RUN 6/10 - acc: 75.08% +- 4.29% - auc: 84.11% +- 3.29%
PPOL RUN 7/10 - acc: 74.26% +- 4.41% - auc: 82.61% +- 1.97%
PPOL RUN 8/10 - acc: 73.16% +- 2.99% - auc: 82.49% +- 2.81%
PPOL RUN 9/10 - acc: 72.44% +- 2.49% - auc: 81.21% +- 2.4%
PPOL RUN 10/10 - acc: 76.16% +- 2.54% - auc: 83.74% +- 2.35%
PPOL GLOBAL - acc: 74.44% +- 1.34% - auc: 83.11% +- 1.02%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
